<h1> DS200A Computer Vision Assignment</h1>

<h2>  Part Four: Extension Activities</h2>	


TensorFlow (Optional)- Now, try using TensorFlow to categorize your images. The accuracy should be significantly higher due to the usage of nueral nets



In [1]:
import tensorflow as tf
import keras
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

Using TensorFlow backend.


In [3]:
train_df = pd.read_pickle("starting_data_part1.pkl").\
drop(labels=["filename", "aspect"], axis = 1)

In [4]:
#Normalize: If a image only has 2 channels, it sets all 3 channels to the 
#1st channel for easier processing. This occurs in about 1% of cases. 
def normalize(image):
    if len(image.shape) == 2:
        temp = np.zeros((image.shape[0], image.shape[1], 3))
        for i in range(3):
            temp[:,:,i] = image
        image = temp
    return image

train_df['image'] = train_df['image'].apply(normalize)

In [5]:
#Transform: Reshapes the array for processing in TensorFlow. 
def transform(arr):
    new_arr = np.zeros((len(arr), 128, 128, 3))
    for i in range(len(arr)):
        new_arr[i] = arr[i]
    return new_arr

In [6]:
#Splits the training dataframe into a train and validation set, and then
#splits those into the respective feature matrices and the labels. 
train, valid = train_test_split(train_df, test_size=0.1, random_state=42,\
                                shuffle=True, stratify=train_df['class'])

train_x, train_y = np.array(train['image']), np.array(train['class'])
train_x = transform(train_x / 255)
valid_x, valid_y = np.array(valid['image']), np.array(valid['class'])
valid_x = transform(valid_x / 255)

In [7]:
# Builds a simple CNN for image recognition, consisting of convolutional layers,
# max pooling layers, and some densely connected layers at the end leading to
# a final layer of 20 neurons that help predict the category. Regularization
# is achieved by Keras kernel_regularizers during the Dense layers, although
# it's hard to get good regularzation with a very small dataset. 
model = keras.Sequential()
model.add(Conv2D(32, (3,3), input_shape = (128, 128, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Conv2D(32, (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Flatten())
model.add(Dense(128, activation = 'relu', kernel_regularizer=
                keras.regularizers.l2(0.001)))
model.add(Dense(64, activation = 'relu', kernel_regularizer=
                keras.regularizers.l2(0.05)))
model.add(Dense(20, activation = 'sigmoid', kernel_regularizer=
                keras.regularizers.l2(0.05)))

model.compile(optimizer=tf.train.AdamOptimizer(), 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(train_x, train_y, batch_size = 64, epochs = 17)

Epoch 1/17
1350/1350 [==============================] - 10s 8ms/step - loss: 8.1601 - acc: 0.1126
Epoch 2/17
1350/1350 [==============================] - 6s 4ms/step - loss: 6.5430 - acc: 0.1652
Epoch 3/17
1350/1350 [==============================] - 6s 5ms/step - loss: 5.4022 - acc: 0.1919
Epoch 4/17
1350/1350 [==============================] - 6s 5ms/step - loss: 4.5676 - acc: 0.2289
Epoch 5/17
1350/1350 [==============================] - 6s 4ms/step - loss: 3.8568 - acc: 0.2926
Epoch 6/17
1350/1350 [==============================] - 6s 5ms/step - loss: 3.3586 - acc: 0.3719
Epoch 7/17
1350/1350 [==============================] - 6s 5ms/step - loss: 3.0068 - acc: 0.4259
Epoch 8/17
1350/1350 [==============================] - 6s 4ms/step - loss: 2.6856 - acc: 0.4844
Epoch 9/17
1350/1350 [==============================] - 6s 4ms/step - loss: 2.4163 - acc: 0.5659
Epoch 10/17
1350/1350 [==============================] - 6s 4ms/step - loss: 2.2013 - acc: 0.6111
Epoch 11/17
1350/1350 [=====

In [8]:
#Evaluates the model on the unseen validations et. 
test_loss, test_acc = model.evaluate(valid_x, valid_y)
print('Test Loss is: ', test_loss)
print('Test Accuracy is: ', test_acc)


151/151 [==============================] - 1s 5ms/step
3.29160676413 0.384105962633
